In [ ]:
import osmnx as ox
import networkx as nx
import random
import matplotlib.pyplot as plt
import heapq

def k_shortest_paths_floyd_warshall(graph, k):
    n = graph.number_of_nodes()
    nodes = list(graph.nodes())
    node_index = {node: i for i, node in enumerate(nodes)}
    
    # Initialize distance matrix with empty lists
    dist = [[[] for _ in range(n)] for _ in range(n)]

    # Populate initial distances from the graph
    for i in range(n):
        for j in range(n):
            if i == j:
                dist[i][j].append((0, [nodes[i]]))
            elif graph.has_edge(nodes[i], nodes[j]):
                weight = graph[nodes[i]][nodes[j]][0]['length']
                dist[i][j].append((weight, [nodes[i], nodes[j]]))

    # Floyd-Warshall algorithm to find k shortest paths
    for k_idx in range(n):
        print(f"Processing intermediate node {k_idx+1}/{n}")
        for i in range(n):
            for j in range(n):
                if i != j and i != k_idx and j != k_idx:
                    new_paths = []
                    for cost1, path1 in dist[i][k_idx]:
                        for cost2, path2 in dist[k_idx][j]:
                            new_cost = cost1 + cost2
                            new_path = path1 + path2[1:]
                            heapq.heappush(new_paths, (new_cost, new_path))

                    # Merge new_paths with existing dist[i][j] and keep top k-shortest paths
                    all_paths = dist[i][j] + new_paths
                    all_paths.sort(key=lambda x: x[0])
                    dist[i][j] = all_paths[:k]

    return dist, nodes, node_index

def visualize_paths(graph, start, goal, paths):
    pos = {node: (data['x'], data['y']) for node, data in graph.nodes(data=True)}

    # Define colors for the paths
    colors = ['red', 'orange', 'purple', 'green', 'cyan', 'magenta', 'yellow', 'black']
    
    # Ensure we have enough colors
    colors = (colors * ((len(paths) // len(colors)) + 1))[:len(paths)]

    for idx, (cost, path) in enumerate(paths):
        fig, ax = plt.subplots(figsize=(20, 20))
        
        # Specify colors for nodes
        node_colors = ['blue' if node != start and node != goal else 'green' if node == start else 'red' for node in graph.nodes()]
        nx.draw(graph, pos, node_size=10, node_color=node_colors, edge_color='gray', ax=ax)
        
        # Draw the path with a specific color
        nx.draw(graph, pos, nodelist=path, node_size=50, node_color=colors[idx], ax=ax)
        nx.draw_networkx_edges(graph, pos, edgelist=list(zip(path, path[1:])), edge_color=colors[idx], width=2, ax=ax)
        
        plt.title(f'Path {idx + 1}: Cost = {cost}')
        plt.show()

def main():
    place_name = "New York"
    dist = 1500

    # Fetch the graph and keep the largest strongly connected component
    graph = ox.graph_from_address(place_name, dist=dist, network_type='drive')
    graph = ox.truncate.largest_component(graph, strongly=True)

    # Convert the OSMnx graph to a NetworkX graph
    nx_graph = nx.MultiDiGraph(graph)

    source = list(nx_graph.nodes())[random.randint(0, len(nx_graph.nodes()) - 1)]
    destination = list(nx_graph.nodes())[random.randint(0, len(nx_graph.nodes()) - 1)]
    while destination == source:
        destination = list(nx_graph.nodes())[random.randint(0, len(nx_graph.nodes()) - 1)]

    print(f"Source: {source}, Destination: {destination}")

    k = 3  # Number of shortest paths to find
    dist, nodes, node_index = k_shortest_paths_floyd_warshall(nx_graph, k)
    
    source_index = node_index[source]
    destination_index = node_index[destination]
    paths = dist[source_index][destination_index]

    if paths:
        for i, (cost, path) in enumerate(paths):
            print(f"Path {i+1}: {' -> '.join(map(str, path))} with cost {cost}")
        visualize_paths(nx_graph, source, destination, paths)
    else:
        print("No path found")

if __name__ == "__main__":
    main()
